In [ ]:
import json
import pandas as pd
import random
import numpy as np

f = open('all_stock.json')
response = json.load(f)
stock = pd.DataFrame.from_dict(response)

f = open('demand.json')
response = json.load(f)
demand = pd.DataFrame.from_dict(response)

MODEL = '45'
FAMILY = 'SHS'
stock = stock[(stock.product_family == FAMILY) & (stock.product_model == MODEL) & (stock.virtual_location == 'GY3') & (stock.product_length < 5500)]
stock = stock.replace(np.nan, 0)
for index, r in enumerate(zip(*stock.to_dict("list").values())):                             
    if r[3] == 0:        
        stock.iloc[index, 3] = int(r[1][6:9]) / 10
        if "BLANK" not in r[0]:
            stock.iloc[index, 6] = 105.0
test = stock.groupby(["product_length", "product_g1"]).first()
test.reset_index(inplace=True)
test = test[test['product_pitch'] != 0]

demand = demand[demand.facility.str.startswith('C0010')]
dem = demand[(demand.product_family == FAMILY) & (demand.product_model == MODEL) & (~demand.item_description.str.contains('(GP)')) & (demand.product_length < 5500)].copy()
dem_group = dem.groupby('product_length').first()
dem_group['qty'] = dem.groupby('product_length')['order_quantity'].sum()
dem_group = dem_group.reset_index()
dem_group.sort_values('qty', inplace=True)

dem_r = dem_group[::-1]
dem_len_list = np.array(dem_r.product_length.tolist(), int)
dem_types = len(dem_len_list)
dem_qty_list = dem_r['qty'].tolist()
dem_g_list = dem_r['product_g1'].tolist()
stock_g_list = test['product_g1'].tolist()
clamp = 10
kerf = 2

def get_g2(length, g1, pitch):
    return (length - g1) % pitch    

def get_possible_quantity(long_len, long_g1, short_len, short_g1, pitch):
    possible_quantity = 0
    loss_at_cut = [0]
    len_at_point = [long_len]    
    while (((short_g1 + clamp + kerf <= long_g1) & ((long_len - long_g1) >= (short_len - short_g1))) | ((long_len - long_g1) > short_len)):
        cut = 1
        front_loss = 0
        if long_g1 == short_g1:            
            cut = 0
        elif short_g1 + clamp + kerf < long_g1:
            front_loss = long_g1 - short_g1                        
        else:
            front_loss = long_g1  + (pitch - short_g1)
        short_g2 = get_g2(short_len, short_g1, pitch) 
        long_g1 = pitch - (short_g2 + kerf)
        possible_quantity += 1
        long_len -= short_len + front_loss + kerf        
        len_at_point.append(long_len)
        loss_at_cut.append(front_loss + kerf * cut) 
    return possible_quantity, len_at_point, loss_at_cut

def generate_arr(stock_types, temp_stock_len_list, r, d_index, longest):
    dem_len = r[0]
    dem_qty = r[-1]
    pitch = r[9]
    arr = np.zeros(stock_types, dtype=int)    
    loss = np.zeros(stock_types, dtype=int)    
    if dem_qty == 0:
        return arr, loss    
    for index, stock_len in enumerate(temp_stock_len_list):        
        possible_quantity, len_at_point, loss_at_cut = get_possible_quantity(stock_len, stock_g_list[index], dem_len, dem_g_list[d_index], pitch)        
        if possible_quantity:
            rand_val = random.randint(0 + int(longest), min(possible_quantity, dem_qty))
            temp_stock_len_list[index] = len_at_point[rand_val]
            arr[index] = rand_val
            loss[index] = sum(loss_at_cut[:(rand_val+1)])
            dem_qty -= rand_val
            # print(stock_len, index, dem_len, loss)
            if dem_qty == 0:
                return arr, loss
        else:
            pass
    return arr, loss
    

In [ ]:
dem_qty_list = np.array(dem_r['qty'].tolist())
stock_qty = np.array([200, 200, 100, 100])
count_ = np.count_nonzero(dem_qty_list)
stock_len_list_original = np.array(test.product_length.tolist(), int) 
map = {}
while count_ > 0:    
    initial_stock_len_list = stock_len_list_original[stock_qty != 0]        
    stock_types = len(initial_stock_len_list)
    best_loss_sum = np.empty(stock_types)
    best_loss_sum.fill(9999)
    best_loss = []
    best = np.zeros((dem_types, stock_types), int)           
    longest = False
    for i in range(count_ * 10):        
        stock_len_list = initial_stock_len_list.copy()        
        arr = np.zeros((0,stock_types), dtype=int)
        loss = np.zeros((0,stock_types), dtype=int)
        for index, r in enumerate(zip(*dem_r.to_dict("list").values())):                             
            if ~longest & int(r[-1]):
                longest = True            
            arr_, loss_ = generate_arr(stock_types, stock_len_list, r, index, longest)            
            arr = np.append(arr, [arr_], axis=0)
            loss = np.append(loss, [loss_], axis=0)                
        remainder = (stock_len_list != initial_stock_len_list) * stock_len_list
        loss = np.append(loss, [remainder], axis=0)
        loss_sum = loss.sum(axis = 0)
        index = best_loss_sum > loss_sum
        best_loss_sum[index] = loss_sum[index]
        best[:, index] = arr[:, index]            
    best_loss_sum[best_loss_sum == 0] = 9999
    stock_i = np.argmin(best_loss_sum)
    best_dem = best[:, stock_i]
    
    dem_i = best_dem != 0    
    res = list(zip(dem_len_list[dem_i], best_dem[dem_i]))   

    non_0 = np.where(best_dem != 0)[0]    
    multiplier = np.min(dem_qty_list[non_0] // best_dem[non_0])
    stock_qty[stock_i] -= 1
    best_dem = best_dem * multiplier
    for index in range(len(res)):    
        dem_len, self_multiplier = res[index]
        tmp = res.copy()
        tmp.pop(index)
        tmp_res = [stock_len_list_original[stock_i], tmp, int(best_loss_sum[stock_i]), self_multiplier, int(multiplier)]
        if dem_len in map.keys():    
            map[dem_len].append(tmp_res)        
        else:
            map[dem_len] = [tmp_res]
    # print(stock_len_list_original[stock_i], res, int(best_loss_sum[stock_i]), int(multiplier))
    dem_r['qty'] = dem_qty_list - best_dem
    dem_qty_list = np.array(dem_r['qty'].tolist())
    count_ = np.count_nonzero(dem_qty_list)
map

In [ ]:
# stockPool = [{'qty': 8, 'len': 4050, 'des': 'HSR45-4050L HALF RAIL 3', 'num': '301745300940500', 'incoming': [], 'incoming_qty': 0, 'total_len': 32}, {'qty': 65, 'len': 3195, 'des': 'HSR45-3195 HALF RAIL 3', 'num': '301745200031950', 'incoming': [], 'incoming_qty': 0, 'total_len': 207}]
# stock_df = pd.DataFrame.from_dict(stockPool)
# stock_df.sort_values('len', inplace=True)
# stock_df['g1'] = stock_df['num'].apply(lambda x:int(x[6:9]) / 10)

demandPool = [{'qty': 1, 'len': 2790, 'des': 'SHS20-2790L RAIL', 'num': '402420150027900', 'date': '14 Dec', 'mos': ['M156505'], 'total_len': 3, 'g1': 15, 'pitch': 60}, {'qty': 40, 'len': 3000, 'des': 'SHS20-3000L(GK) RAIL FOR CUT-S', 'num': '302451022774900', 'date': '16 Dec', 'mos': ['M172105', 'M172103', 'M172102', 'M172104'], 'total_len': 120, 'g1': 20, 'pitch': 60}]
demand_df = pd.DataFrame.from_dict(demandPool)
dem_group = demand_df.groupby('len').first()
dem_group['qty'] = demand_df.groupby('len')['qty'].sum()
dem_group = dem_group.reset_index()
dem_group.sort_values('qty', inplace=True)
dem_r = dem_group[::-1]
dem_len_list = np.array(dem_r.len.tolist(), int)
dem_types = len(dem_len_list)
dem_qty_list = dem_r['qty'].tolist()
dem_g_list = dem_r['g1'].tolist()
dem_r



In [ ]:
df = pd.DataFrame(best * dem, columns = test.product_length.tolist())
df.index = dem_len_list
df['qty'] = dem_r.qty.tolist()
df_sum = df.sum()
arr2 = np.array(test.product_length.tolist(), int) - df_sum.values[:-1]
arr2 = np.append(arr2, 156)
df.append(pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df)))



In [ ]:
dem_len_list = np.array(dem_group.product_length.tolist(), int)
stock_len_list = np.array(test.product_length.tolist(), int)

def numbers_with_sum(stock_index, length, n, k):    
    s = stock_len_list[stock_index]
    if k > 0:
        if s < length:
            num = 0            
        else:        
            num = random.randint(0, s // length)
    else:
        num = 0
    if n == 1:
        return [k]    
    return [num] + numbers_with_sum(stock_index+1, length, n - 1, k - num)

In [ ]:
test

In [ ]:
# arr = np.empty((0,len(dem_len_list)), int)
print(stock_len_list)
for d in dem_len_list:
    print(d, [s % d for s in stock_len_list])
    # arr = np.append(arr, np.array([r % x for x in dem_len_list]), axis=0)
arr

In [ ]:
dem_r = dem_group[::-1]
dem_len_list = np.array(dem_r.product_length.tolist(), int)
dem_types = len(dem_len_list)
dem_qty_list = dem_r['qty'].tolist()

In [ ]:
total_sum = 0.0
for f in range(50):
    best = []
    min_ = 10000
    loss_sum_min = []
    dem_qty_list = dem_r['qty'].tolist()        
    count_ = np.count_nonzero(dem_qty_list)
    print(count_)
    for i in range(count_ * 100):
        stock_len_list = np.array(test.product_length.tolist(), int)
        stock_types = len(stock_len_list)
        dem = np.repeat([dem_len_list], 4, axis=0).T
        arr = np.zeros((0,stock_types), dtype=int)

        def numbers_with_sum(dem_index, dem_len, dem_qty):
            arr = np.zeros(stock_types, dtype=int)
            for index, stock_len in enumerate(stock_len_list):
                possible_quantity = stock_len // dem_len
                if possible_quantity:
                    # if stock_len_list[index + 1] & 1:
                    #     pass
                    rand_val = random.randint(0, min(possible_quantity, dem_qty))
                    stock_len_list[index] -= rand_val * dem_len
                    arr[index] = rand_val
                    dem_qty -= rand_val
                    if dem_qty == 0:
                        return arr                    
                else:
                    pass        
            return arr                
        for index, r in enumerate(zip(*dem_r.to_dict("list").values())):                             
            arr = np.append(arr, np.array([numbers_with_sum(index, r[0], r[-1])]), axis=0)        
        loss_sum = np.array(test.product_length.tolist(), int) - (arr * dem).sum(axis=0)
        cur_loss = (loss_sum).sum()
        # cur_sum = (arr * dem).sum()    
        if cur_loss < min_:
            best = arr
            min_ = cur_loss    
    dem_r['qty'] = dem_qty_list - best.sum(axis=1)
    df = pd.DataFrame(best * dem, columns = test.product_length.tolist())
    df.index = dem_len_list
    df['qty'] = dem_r.qty.tolist()
    df_sum = df.sum()
    arr2 = np.array(test.product_length.tolist(), int) - df_sum.values[:-1]
    total_sum += arr2.sum()
    arr2 = np.append(arr2, 156)
    pd.concat([df, pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df))])

    # df.append(pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df)))    
    print(df)
    # print(total_sum)